
# General imports

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
import requests
from io import BytesIO

# Data importeren

## Bruggendata

In [ ]:
# Lees de dexter export van brugdata in
brug = pd.read_csv('https://dqpublicblob.blob.core.windows.net/data-quality/SVM-model/brugdata/sb-bruggen-export-aalsmeerderbrug.csv',
                 low_memory=False)


In [ ]:
# Kaart van de brug
url = 'https://dqpublicblob.blob.core.windows.net/data-quality/SVM-model/brugdata/Geselecteerde%20locaties.png'

# Fetch the image
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# Display the image
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
# Zorg dat de opening en sluitingstijden echte datum/tijd velden zijn
brug['opening'] = pd.to_datetime(brug['geopend'])
brug['sluiting'] = pd.to_datetime(brug['gesloten'])


## Reistijden

In [ ]:
# Lees de export van reistijden van 4 trajecten vanuit Dexter in.
rt = pd.read_csv('https://dqpublicblob.blob.core.windows.net/data-quality/SVM-model/reistijden/reistijd-export-aalsmeerderbrug-volledig.csv',
                 low_memory=False)


In [ ]:
# Omdat de FCD en reistijd een vertraging heeft, verschuiven we starttijd met 6 minuten.
rt['start'] = pd.to_datetime(rt['start_meetperiode']) - pd.to_timedelta(6, 'min')

# Beschouw alleen de periode tussen 6-21 uur. Dan is er in ieder geval genoeg FCD.
rt = rt[(rt['start'].dt.hour > 6) & (rt['start'].dt.hour < 21)]

# Bekijk de eerste 5 records eens.
rt.head(5)

## Bewerk de reistijden
De reistijden worden geleverd in losse records voor elk traject.
Voor de analyse willen we dat de trajectdata in kolommen staat, zodat we voor elke minuut
de data kunnen gebruiken. Daarom pivoteren we de gegevens.

In [ ]:
pivoted_travel_times = rt.pivot_table(index='start',
                                        columns='id_meetlocatie',
                                        values=['gem_reistijd'  #,
                                                #'kwaliteitsindicator_reistijd',
                                                #'waarnemingen_reistijd'
                                                ]
                                      )

### Nu gaan we de 'ground thruth' data toevoegen waarop we het model trainen.

In [ ]:
# Voeg een kolom 'brug_open' toe
pivoted_travel_times['brug_open'] = 0

# Markeer reistijden die tijdens een brugopening vallen
# We kennen een opening de waarde 10 toe, zodat we die later makkelijker kunnen visualiseren.
for idx, row in brug.iterrows():
    mask = (pivoted_travel_times.index >= pd.to_datetime(row['opening'])) & (
            pivoted_travel_times.index <= pd.to_datetime(row['sluiting']))
    pivoted_travel_times.loc[mask, 'brug_open'] = 10

In [ ]:
# Laten we de gegevens in een grafiek tonen, zodat we wat inzicht hebben in waar we naar kijken.
pivoted_travel_times.plot.line(y=pivoted_travel_times.columns, color =['red', 'blue', 'orange',
                                                                       'grey', 'black'])
plt.show()


# Maak ook een interactieve plot.

fig = go.Figure()
cols = pivoted_travel_times.columns
color =['red', 'blue', 'orange', 'grey', 'black', 'purple']
for col in range(len(cols)):
    y_data = pivoted_travel_times[cols[col]]
    name = str(cols[col])
        
    fig.add_trace(go.Scatter(x=pivoted_travel_times.index, y=y_data, mode='lines', name=name, line=dict(color=color[col])))

fig.write_html('SVM-input-data-kennisevent.html')


Om de interactieve plot te bekijken, kan je links klikken op het mapje (bestanden) en eventueel de inhoud vernieuwen (het cirkel-pijltje).
Download het bestand en open dit door er dubbel op te klikken. Je kunt nu lijnen uit en aan zetten en inzoomen.

# Bouwen van het model

## Data voorbereiding

In [ ]:
# Selecteer de relevante features (alle kolommen) en de target variabele
# Features zijn de waarden die we gebruiken om de target te voorspellen.

features = pivoted_travel_times.columns.drop('brug_open')
target = 'brug_open'

In [ ]:
# Nu bepalen we de data waarop we trainen. Dit zijn de gegevens van 29/5 tot 3/6.
traindata = pivoted_travel_times[pivoted_travel_times.index < '2024-06-03 00:00:00']
X = traindata[features]
y = traindata[target]

In [ ]:
# Splits de data in trainings- en testsets
# De 80/20 regel is vaak goed. En we testen over dezelfde periode als de trainingsdata.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

## Resampling en normaliseren

Nu is er een probleem. De brug is veel vaker dicht (horizontaal) dan open (verticaal). Dus als we er niets aan doen, trainen we meer op een juiste dicht voorspelling. Dat willen we niet.

Een tweede probleem is dat de waardes van de features sterk uiteenlopen in grootte. Hiervoor kan je corrigeren door ze te normaliseren.


### Resampling

In [ ]:
# Combineer X_train en y_train voor het resampling proces
train_data = pd.concat([X_train, y_train], axis=1)

# Scheid majority en minority classes
not_open = train_data[train_data['brug_open'] == 0]
open = train_data[train_data['brug_open'] == 10]

# Resample de minority class
open_upsampled = resample(open,
                          replace=True,  # sample with replacement
                          n_samples=len(not_open),  # match number in majority class
                          random_state=42)  # reproducible results

# Combineer majority en upsampled minority class
upsampled = pd.concat([not_open, open_upsampled])

# Scheid de features en de target
X_train_balanced = np.array(upsampled.drop('brug_open', axis=1))
y_train_balanced = np.array(upsampled['brug_open'])

# Print het aantal voorkomens voor brug gesloten(0) en brugopen(10)
print(np.unique(y_train_balanced, return_counts=True))

### Normaliseren

In [ ]:
# Normaliseer de features.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_balanced = scaler.fit_transform(X_train_balanced)

## Model definitie en training
We kiezen voor een Support Vector Machine classificatie model met een RBF kernel. Deze is het meest flexibel.

Als je meer wilt weten: https://www.geeksforgeeks.org/radial-basis-function-kernel-machine-learning/

In onderstaande afbeelding zie je waarom een Radial Basis Kernel (RBF) in dit geval waarschijnlijk het beste presteert:

In [ ]:
url = 'https://dqpublicblob.blob.core.windows.net/data-quality/SVM-model/SVM-kernels.jpg'

# Fetch the image
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# Display the image
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.show()


In [ ]:
# Kies een SVM met een RBF kernel
svm_model = SVC(kernel='rbf', gamma='auto')  # auto

# Train het model
svm_model.fit(X_train_balanced, y_train_balanced)

## Model testen

In [ ]:
# Voorspel de test data
y_pred = svm_model.predict(X_test)


# Genereer een classificatierapport en een verwarringsmatrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# https:/en.wikipedia.org/wiki/Precision_and_recall
url = 'https://dqpublicblob.blob.core.windows.net/data-quality/SVM-model/Precisionrecall.png'

# Fetch the image
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# Display the image
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.show()

We hebben nu een lage precissie, maar een hoge recall voor brug open (10).

We willen graag dat wanneer een brug echt open is, dat we zo min mogelijk zeggen dat die gesloten is. Dit is recall.

Maar misschien is het niet zo heel erg als we zeggen dat brug open is, terwijl deze gesloten is (pressision). Mogelijk dat dit alleen rond de randen gebeurd. We zullen zien.


# Voorspelling over de gehele periode
We gaan de brugopeningen voorspellen over de gehele periode, dus ook over 4,5 en 6 juni.

In [ ]:
pivoted_travel_times['prediction'] = svm_model.predict(
    scaler.transform(pivoted_travel_times[features])) * -1


Laten we dit weer in een  interactieve grafiek tonen.

In [ ]:
fig = go.Figure()
cols = pivoted_travel_times.columns
color =['red', 'blue', 'orange', 'grey', 'black', 'purple', 'green']
for col in range(len(cols)):
    y_data = pivoted_travel_times[cols[col]]
    name = str(cols[col])
    fig.add_trace(go.Scatter(x=pivoted_travel_times.index, y=y_data, mode='lines', name=name, line=dict(color=color[col])))

fig.write_html('SVM-predictie-kennisevent.html')

Waardoor zou de afwijking kunnen komen op 6 juni?